feature_engineering.py
self.data_frame.drop("YearMonth")
correlation_matrix = self.data_frame.corr()
relevant_f = correlation_matrix["reservation_status_Canceled"].sort_values(
  ascending=False
)

relevant_f.to_excel("correlation_matrix.xlsx")

In [ ]:
# merge.py
import os
import jupytext

In [ ]:
# Directory containing Python files
directory = "."
notebook_content = ""

In [ ]:
# Loop through all Python files in the directory and merge their content
for filename in os.listdir(directory):
    if filename.endswith(".py"):
        filepath = os.path.join(directory, filename)
        with open(filepath, "r") as file:
            notebook_content += (
                f"# {filename}\n"  # Add the file name as a markdown cell
            )
            notebook_content += file.read() + "\n\n"  # Add the content of the file

In [ ]:
# Save the merged content as a single notebook
notebook = jupytext.reads(notebook_content, fmt="py")
jupytext.write(notebook, "merged_notebook.ipynb")

In [ ]:
# plots.py
"""
@brief:
Provides methods to plot data. The methods are data agnostic for reuse.

@author: Alain Christian (U2083264)
"""

In [ ]:
import matplotlib.pyplot as pylt
import seaborn as sns
import plotly.express as px
import geopandas as gpd

In [ ]:
from pandas import DataFrame
import numpy as np
import random

In [ ]:
class Charts:
    """
    This class houses methods for creating charts.
    """

    def __init__(self):
        pass

    def bar_chart(
        self,
        values: list,
        properties: list,
        y_axis_lbl: str,
        title: str,
        x_axis_lbl: str = None,
    ):
        """
        Creates a bar chart using a list of provided values and properties.
        """
        fig, ax = pylt.subplots()
        labels = properties
        colors = [
            (random.random(), random.random(), random.random(), 1) for value in values
        ]
        ax.bar(properties, values, label=labels, color=colors)
        ax.set_ylabel(y_axis_lbl)
        ax.set_xlabel(x_axis_lbl)
        ax.legend(title=title)

        return pylt

    def line_chart(self, properties: list, values: list, labels: list[str], title: str):
        """
        Creates a line chart using a list of provided values and properties.
        """
        # Create a line chart
        pylt.figure(figsize=(8, 6))
        pylt.plot(properties, values, marker="o", linestyle="-")
        pylt.title(title)
        pylt.xlabel(labels[0])
        pylt.ylabel(labels[1])
        pylt.grid(True)

    def pie_chart(self, properties: list, values: list, title: str):
        """
        Creates a pie chart using a list of provided values
        """
        colors = [
            (random.random(), random.random(), random.random(), 1) for value in values
        ]

        # Create a pie chart
        fig, ax = pylt.subplots(figsize=(15, 15))
        ax.pie(
            values,
            labels=properties,
            colors=colors,
            autopct="%1.1f%%",
            startangle=90,
            pctdistance=0.85,
            labeldistance=1.5,
        )
        pylt.legend(properties, loc="center left", bbox_to_anchor=(1, 0.5), ncol=3)
        pylt.title(title)

In [ ]:
class Maps:
    """
    This class presents methods that create various types of maps
    """

    def __init__(self):
        pass

    def heat_map(self, contingency_tbl, annot=True, cmap="YlGnBu"):
        """
        Creates a heat map with the provided contigency
        """
        pylt.imshow(contingency_tbl, cmap, aspect="auto")
        pylt.colorbar()
        pylt.xticks(np.arange(contingency_tbl.shape[1]), contingency_tbl.columns)
        pylt.yticks(np.arange(contingency_tbl.shape[0]), contingency_tbl.index)
        return pylt

    def choropleth_map(self, df: DataFrame):
        """
        Returs a choropleth map
        """
        url = "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"
        world = gpd.read_file(url)
        world.to_excel("output.xlsx")
        # Merge dataset / frame to add geospatial data

        merged_frame = world.merge(df, how="left", left_on="ISO_A3", right_on="country")
        merged_frame["country"].dropna(inplace=False)
        merged_frame.to_excel("country.xlsx")
        fig, ax = pylt.subplots(1, 1, figsize=(14, 10))

        merged_frame.plot(
            column="count",
            ax=ax,
            legend=True,
            cmap="YlGnBu",
            missing_kwds={
                "color": "lightgrey",
                "edgecolor": "red",
                "hatch": "/\/",
                "label": "Unknown values",
            },
        )

        ax.set_title("Bookings per country")
        ax.set_xlabel("Longitude")
        ax.set_ylabel("Latitude")
        pylt.legend(loc="lower right")

        ax.set_axis_off()

In [ ]:
# insights.py
"""
@brief:
This module uses the pre-processed dataframe from the cleaner
to derive insights from the data.

@author: Alain Mugisha (u2083264)
"""

In [ ]:
from cleaner import Cleaner
from typing import List
from plots import Charts, Maps, pylt
from pandas import DataFrame, Series, crosstab
from typing import cast

In [ ]:
class Insights:
    """
    Produces insights based on the passed information.
    """

    def __init__(self):
        self.cleaner = Cleaner()
        self.data_frame = self.cleaner.validate_data()

    def cancellation_percentage_per_hotel(self) -> List:
        """
        Calculates the percentage of cancellation per city
        """
        bookings_grouped = self.data_frame.groupby("hotel")["hotel"].count()
        bookings_city_canceled = (
            self.data_frame[
                (self.data_frame["is_canceled"] == True)
                & (self.data_frame["hotel"] == "City Hotel")
            ].size
            / 32
        )
        bookings_resort_canceled = (
            self.data_frame[
                (self.data_frame["is_canceled"] == True)
                & (self.data_frame["hotel"] == "Resort Hotel")
            ].size
            / 32
        )
        resort_perc = (
            bookings_resort_canceled / bookings_grouped["Resort Hotel"]
        ) * 100
        city_perc = (bookings_city_canceled / bookings_grouped["City Hotel"]) * 100

        return [resort_perc, city_perc]

    def most_ordered_meals(self) -> DataFrame:
        """
        Returns a data frame of the most ordered meal types.
        """
        meals = self.data_frame.groupby("meal")["meal"].count()
        return cast(DataFrame, meals)

    def most_booked_room_types(self) -> Series:
        """
        Returns a list of the most booked meals
        """
        # The focus is on the reserved room because at the time of
        # booking this is the room type given until it is potentially changed.
        room_type = self.data_frame.groupby("reserved_room_type")[
            "reserved_room_type"
        ].count()
        return cast(Series, room_type)

    def most_common_customer_types(self):
        """
        Returns a list of the most common customer types
        """
        # The focus is on the reserved room because at the time of
        # booking this is the room type given until it is potentially changed.
        customer_type = self.data_frame.groupby("customer_type")[
            "customer_type"
        ].count()
        return customer_type

    def get_returning_guests(self) -> float:
        """
        Returns a number of returning guests
        """
        r_guests = (
            self.data_frame[(self.data_frame["is_repeated_guest"] == True)].size / 32
        )
        return r_guests

    def correlation_between_columns(self, index, columns: list):
        """
        Returns a heat map showing the correlation between a passed list of columns
        It uses cross tabulation to show the frequency distribution of x amount of columns
        """
        contingency_tbl = crosstab(
            index=self.data_frame[index],
            columns=[self.data_frame[col] for col in columns],
            normalize="index",
        )
        return contingency_tbl

In [ ]:
if __name__ == "__main__":
    insight = Insights()
    plots = Charts()
    maps = Maps()

    # Printing information about the percentage of cancellations per hotel.
    values = insight.cancellation_percentage_per_hotel()
    properties = ["% City Hotel", "% Resort Hotel"]
    y_axis_label = "Percentage of cancellation"
    title = "Percentage of cancellation per type of hotel"
    plt_canc = plots.bar_chart(values, properties, y_axis_label, title)

    # Printing information about the most ordered meals.
    values = insight.most_ordered_meals().values.tolist()
    properties = ["BB", "FB", "HB", "SC", "Undefined"]
    y_axis_label = "Meal orders per meal type"
    title = "Most ordered meals per type"
    plt_meals = plots.bar_chart(values, properties, y_axis_label, title)

    # Printing information about the most sought after room types
    values = insight.most_booked_room_types().values.tolist()
    properties = ["A", "B", "C", "D", "E", "F", "G", "H", "L", "P"]
    y_axis_label = "Number of bookings"
    title = "Most sought after rooms"
    plt_rooms = plots.bar_chart(values, properties, y_axis_label, title)
    # plt_rooms.show()

    # Printing information about the most common type of customer
    values = insight.most_common_customer_types().values.tolist()
    properties = ["Contract", "Group", "Transient", "Transient-Party"]
    y_axis_label = "Number of bookings"
    title = "Most common type of customers"
    plt_customer_types = plots.bar_chart(values, properties, y_axis_label, title)
    # plt_rooms.show()

    # Printing the number of the returning customers
    returning_customers = insight.get_returning_guests()

    # Showing correlation between room types and cancellations
    # With a plotted crosstab in a heatmap, crosstab because it will allow us to view
    # The frequency distribution between the two columns

    columns = ["reserved_room_type", "assigned_room_type"]
    cont_tbl = insight.correlation_between_columns("is_canceled", columns)
    cont_tbl.to_excel("cont_table.xlsx", sheet_name="Crosstab Data")
    pylt.show()

In [ ]:
# training.py
"""
@brief:
Provides training functionality for the cleaned and prepared
data presented as a dataframe
"""

In [ ]:
from pandas.core.common import random_state
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
from cleaner import Cleaner

In [ ]:
class Train:
    """
    Provides methods to facilitate the training
    """

    def __init__(self) -> None:
        """
        Base constructor, setting up class properties
        """
        self.data_frame = Cleaner().data_frame

    def split(self):
        """
        Splits the dataset into features and labels
        """
        # print(self.data_frame.columns)
        x = self.data_frame[
            [
                "stays_in_week_nights",
                "stays_in_weekend_nights",
                "previous_cancellations",
                "previous_bookings_not_canceled",
                "booking_changes",
                "days_in_waiting_list",
                "is_repeated_guest",
                "lead_time",
                "deposit_type_Non Refund",
                "adr",
                "assigned_room_type_A",
                "agent",
                "customer_type_Transient",
                "market_segment_Groups",
                "country_PRT",
                "hotel_City Hotel",
                "hotel_Resort Hotel",
                "meal_FB",
            ]
        ]
        y = self.data_frame["reservation_status_Canceled"]
        x_train, x_test, y_train, y_test = train_test_split(
            x, y, test_size=0.3, random_state=42
        )

        return (x_train, x_test, y_train, y_test)

    def train(self):
        """
        Method for training the dataset using RandomForestClassifier
        """
        x_train, x_test, y_train, y_test = self.split()
        rf = RandomForestClassifier(n_estimators=100, random_state=42)

        rf.fit(x_train, y_train)

        y_pred = rf.predict(x_test)

        accuracy = accuracy_score(y_test, y_pred)
        print(f"Accuracy: {accuracy: .2f}")

In [ ]:
if __name__ == "__main__":
    train = Train()
    train.train()

In [ ]:
# eda.py
"""
@brief:
Provides modules to do exploratory data analysis on given dataframes.
@author: Alain Mugisha(U2083264)
"""

In [ ]:
from pandas import DataFrame, Series
from matplotlib import pyplot as pylt
from typing import List
from plots import Charts, Maps
from cleaner import Cleaner

In [ ]:
class EDA:
    """
    Provides methods for carrying out Exploratory analysis
    on a variables
    """

    def __init__(self):
        """
        Constructor method
        """
        self.charts = Charts()
        self.maps = Maps()
        self.cleaner = Cleaner()
        self.data_frame = self.cleaner.data_frame

    def guests_each_month(self):
        """
        Provides of a graphical representation
        of the number of guests per Month/Year combination
        """
        grouped = (
            self.data_frame.groupby("YearMonth")["YearMonth"]
            .count()
            .reset_index(name="count")
        )
        return grouped

    def duration_of_stays(self):
        """
        Provides a dataframe representing the grouped duration of stays.
        """
        grouped = (
            self.data_frame.groupby("duration")["duration"]
            .count()
            .reset_index(name="count")
        )

        return grouped

    def get_geographical_origins(self):
        """
        Provides a dataframe for charting country origins
        """
        grouped = (
            self.data_frame.groupby("country")["country"]
            .count()
            .reset_index(name="count")
        )

        return grouped

In [ ]:
if __name__ == "__main__":
    eda = EDA()

    # Graphing the bookings per month over the years
    df_guests_each = eda.guests_each_month()
    properties = df_guests_each["YearMonth"].values.tolist()
    values = df_guests_each["count"].values.tolist()
    labels = ["Year/Month", "Number of Bookings"]
    title = "Bookings each month"
    eda.charts.line_chart(properties, values, labels, title)

    # Graphing the duration of guest stays in a barchart.
    df_duration = eda.duration_of_stays()
    properties = df_duration["duration"].values.tolist()
    values = df_duration["count"].values.tolist()
    y_lbl_duration = "Stays"
    x_lbl_duration = "Duration of stays"
    title = "Duration of guest stays"
    eda.charts.bar_chart(values, properties, y_lbl_duration, title, x_lbl_duration)

    df_geo_origins = eda.get_geographical_origins()
    properties = df_geo_origins["country"].values.tolist()
    values = df_geo_origins["count"].values.tolist()
    title = "Guest country of origin"
    labels = ["country", "count"]
    eda.maps.choropleth_map(df_geo_origins)

    pylt.show()

In [ ]:
# cleaner.py
"""
@brief:
This module is responsible for cleaning the data.
It will remove any inconsistencies in the data and make it ready for analysis.
@author: Alain Mugisha(U2083264)

"""

In [ ]:
import pandas as pd
from pydantic import BaseModel, ValidationError
from pandas import DataFrame
import calendar

In [ ]:
class Booking(BaseModel):
    """
    This class is the base class modeled based on the data in the csv file.
    """

    hotel: str
    is_canceled: int
    lead_time: int
    arrival_date_year: int
    arrival_date_month: str
    arrival_date_week_number: int
    arrival_date_day_of_month: int
    stays_in_weekend_nights: int
    stays_in_week_nights: int
    adults: int
    children: int
    babies: int
    meal: str
    country: str
    market_segment: str
    distribution_channel: str
    is_repeated_guest: int
    previous_cancellations: int
    previous_bookings_not_canceled: int
    reserved_room_type: str
    assigned_room_type: str
    booking_changes: int
    deposit_type: str
    agent: int
    company: int
    days_in_waiting_list: int
    customer_type: str
    adr: float
    required_car_parking_spaces: int
    total_of_special_requests: int
    reservation_status: str
    reservation_status_date: str

In [ ]:
class Cleaner:
    """
    This is class has as purpose to define and use all multiple classes.
    """

    def __init__(self):
        self.data_frame: DataFrame = pd.read_csv("hotel_bookings.csv")
        months = list(calendar.month_name)[1:]

        """
        Variable alterations
        """
        # Conversion to number format for easy graphical representation
        self.data_frame["arrival_date_month"] = self.data_frame[
            "arrival_date_month"
        ].map(lambda m: months.index(m) + 1)

        self.data_frame["arrival_date_year"] = self.data_frame["arrival_date_year"].map(
            lambda y: str(y).split("20")[1]
        )
        # This was joined to give more context on date sensitive answers.
        self.data_frame["YearMonth"] = (
            self.data_frame["arrival_date_year"].astype(str)
            + "/"
            + self.data_frame["arrival_date_month"].astype(str)
        )

        # This is to be able to use the duration.
        self.data_frame["duration"] = (
            self.data_frame["stays_in_weekend_nights"]
            + self.data_frame["stays_in_week_nights"]
        )
        self.data_frame = pd.get_dummies(
            self.data_frame,
            columns=[
                "customer_type",
                "assigned_room_type",
                "deposit_type",
                "reservation_status",
                "meal",
                "hotel",
                "arrival_date_month",
                "country",
                "market_segment",
                "distribution_channel",
                "reserved_room_type",
                "reservation_status_date",
                "YearMonth",
            ],
        )

        # print(relevant_f)

    def validate_row(self, row) -> Booking | None:
        """
        Validates each row in the dataframe against the Booking class.
        """
        try:
            booking = Booking(**row.to_dict())
            return booking
        except ValidationError as error:
            print(f"Error validating row: {error}")
            return None

    def validate_data(self) -> DataFrame:
        """
        Validates the whole frame and returns a valid data frame cleaned up.
        """
        # Perform a fill based on the mean value
        self.data_frame["children"] = self.data_frame["children"].fillna(
            self.data_frame.groupby("children")["children"].transform("mean")
        )

        # Perform a text filling as the datatype is text
        self.data_frame["country"] = self.data_frame["country"].fillna("N/A")

        # Perform a backward fill to allow data at the top to be filled
        self.data_frame["agent"] = self.data_frame["agent"].bfill()
        self.data_frame["company"] = self.data_frame["company"].bfill()

        # Perform a backward fill to allow data from the bottom to be filled
        self.data_frame["agent"] = self.data_frame["agent"].ffill()
        self.data_frame["company"] = self.data_frame["company"].ffill()

        dropped = self.data_frame.dropna()
        valid_series = dropped.apply(self.validate_row, axis=1)
        valid_df = DataFrame([booking.dict() for booking in valid_series])
        return valid_df

In [ ]:
if __name__ == "__main__":
    cleaner = Cleaner()
    cleaner.validate_data()
    # print(cleaner.validate_data())